In [23]:
import pandas as pd
import geopandas as gpd

# 1. Importing the data
# Load the charging station data, specifying the correct delimiter ';'
df_ladestationen = pd.read_csv("../02_data/01_original_data/ladestationFactTable.csv", sep= ";")
# Load the charging point data
df_ladepunkte = pd.read_csv("../02_data/01_original_data/ladepunktFactTable.csv", sep= ";")
# Load the Shapefile of the German districts using geopandas
# The file path must point to your local .shp file
gdf_landkreise = gpd.read_file('../02_data/02_meta_data/vg250_01-01.gk3.shape.ebenen/vg250_ebenen_0101/VG250_KRS.shp')


# 2. Merging the data
# Join the two dataframes on the common column 'ladestation_id'
df_combined = pd.merge(df_ladestationen, df_ladepunkte, on='ladestation_id', how='left')
df_combined['ladepunkt_id'] = df_combined.index + 1

# 3. Saving the merged data
# Save the combined dataframe to a new CSV file
df_combined.to_csv('../02_data/03_computed_data/combined_ladestation_ladepunkt.csv', index=False)

In [24]:
df_combined.info()
df_combined.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171805 entries, 0 to 171804
Data columns (total 41 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   ladestation_id                 171805 non-null  int64  
 1   Betreiber                      171805 non-null  object 
 2   BetreiberBereinigt             171803 non-null  object 
 3   BetreiberKategorie             171805 non-null  object 
 4   Strasse                        171805 non-null  object 
 5   Hausnummer                     171805 non-null  object 
 6   Adresszusatz                   29126 non-null   object 
 7   PLZ                            171805 non-null  int64  
 8   Ort                            171805 non-null  object 
 9   Bundesland                     171805 non-null  object 
 10  KreisKreisfreieStadt           171805 non-null  object 
 11  Gemeinde                       171805 non-null  object 
 12  ARS                           

,ladestation_id,Betreiber,BetreiberBereinigt,BetreiberKategorie,Strasse,Hausnummer,Adresszusatz,PLZ,Ort,Bundesland,...,CSS_Stecker_Combo_2_Stecker,Tesla_Supercharger_V2_Stecker,Tesla_Supercharger_V3_Stecker,CHAdeMO_Stecker,LPArtLSV,LPArtHPC,LPArtFR,LPArtCharIN,Datenstand_y,ladepunkt_id
0,1,Albwerk Elektro- und Kommunikationstechnik GmbH,Albwerk Elektro- und Kommunikationstechnik,Sonstiges,Ennabeurer Weg,0,NaN,72535,Heroldstatt,Baden-Württemberg,...,False,False,False,False,NLP,NLP (<=22),NLP (<=22),"NLP(>3,7 & <=22)",2025-08-26,1
1,1,Albwerk Elektro- und Kommunikationstechnik GmbH,Albwerk Elektro- und Kommunikationstechnik,Sonstiges,Ennabeurer Weg,0,NaN,72535,Heroldstatt,Baden-Württemberg,...,False,False,False,False,NLP,NLP (<=22),NLP (<=22),"NLP(>3,7 & <=22)",2025-08-26,2
2,2,smopi® - Multi Chargepoint Solution GmbH,smopi - Multi Chargepoint Solution,Sonstiges,Albstraße,15,NaN,72535,Heroldstatt,Baden-Württemberg,...,False,False,False,False,NLP,NLP (<=22),NLP (<=22),"NLP(>3,7 & <=22)",2025-08-26,3
3,2,smopi® - Multi Chargepoint Solution GmbH,smopi - Multi Chargepoint Solution,Sonstiges,Albstraße,15,NaN,72535,Heroldstatt,Baden-Württemberg,...,False,False,False,False,NLP,NLP (<=22),NLP (<=22),"NLP(>3,7 & <=22)",2025-08-26,4
4,2,smopi® - Multi Chargepoint Solution GmbH,smopi - Multi Chargepoint Solution,Sonstiges,Albstraße,15,NaN,72535,Heroldstatt,Baden-Württemberg,...,False,False,False,False,NLP,NLP (<=22),NLP (<=22),"NLP(>3,7 & <=22)",2025-08-26,5


In [25]:
df_combined.shape

(171805, 41)

# 3. Join Chargig Data with Geo Data

In [26]:
# 1. Convert the combined dataframe to a GeoDataFrame
# Create geometries from the latitude and longitude columns
gdf_combined = gpd.GeoDataFrame(
    df_combined, 
    geometry=gpd.points_from_xy(df_combined.Laengengrad, df_combined.Breitengrad)
)

# 2. Set and align the Coordinate Reference System (CRS)
# Set the initial CRS for the charging station points to WGS84 (a common standard for lat/lon)
gdf_combined.set_crs(epsg=4326, inplace=True)

# Transform the CRS of the points to match the CRS of the district polygons
gdf_combined = gdf_combined.to_crs(gdf_landkreise.crs)

# 3. Perform the spatial join
# Match each charging point to the district it is 'within'
gdf_final_join = gpd.sjoin(gdf_combined, gdf_landkreise, how="inner", predicate='within')

# 4. Save the final merged data
# The resulting file now contains charging station data enriched with district information
gdf_final_join.to_csv('../02_data/03_computed_data/final_ladestation_ladepunkt_mit_kreis.csv', index=False)

# Display the first few rows of the final result
print("Preview of the final spatially joined data:")
print(gdf_final_join.head())

Preview of the final spatially joined data:
   ladestation_id                                        Betreiber  \
0               1  Albwerk Elektro- und Kommunikationstechnik GmbH   
1               1  Albwerk Elektro- und Kommunikationstechnik GmbH   
2               2         smopi® - Multi Chargepoint Solution GmbH   
3               2         smopi® - Multi Chargepoint Solution GmbH   
4               2         smopi® - Multi Chargepoint Solution GmbH   

                           BetreiberBereinigt BetreiberKategorie  \
0  Albwerk Elektro- und Kommunikationstechnik          Sonstiges   
1  Albwerk Elektro- und Kommunikationstechnik          Sonstiges   
2          smopi - Multi Chargepoint Solution          Sonstiges   
3          smopi - Multi Chargepoint Solution          Sonstiges   
4          smopi - Multi Chargepoint Solution          Sonstiges   

          Strasse Hausnummer Adresszusatz    PLZ          Ort  \
0  Ennabeurer Weg          0          NaN  72535  Heroldstatt

In [27]:
import pandas as pd

# Define the file path to your newly created CSV file
file_path = '../02_data/03_computed_data/final_ladestation_ladepunkt_mit_kreis.csv'

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)

# Display the first 5 rows of the loaded DataFrame to check its content
print("Successfully loaded the data. Here is a preview:")
print(df.head())

Successfully loaded the data. Here is a preview:
   ladestation_id                                        Betreiber  \
0               1  Albwerk Elektro- und Kommunikationstechnik GmbH   
1               1  Albwerk Elektro- und Kommunikationstechnik GmbH   
2               2         smopi® - Multi Chargepoint Solution GmbH   
3               2         smopi® - Multi Chargepoint Solution GmbH   
4               2         smopi® - Multi Chargepoint Solution GmbH   

                           BetreiberBereinigt BetreiberKategorie  \
0  Albwerk Elektro- und Kommunikationstechnik          Sonstiges   
1  Albwerk Elektro- und Kommunikationstechnik          Sonstiges   
2          smopi - Multi Chargepoint Solution          Sonstiges   
3          smopi - Multi Chargepoint Solution          Sonstiges   
4          smopi - Multi Chargepoint Solution          Sonstiges   

          Strasse Hausnummer Adresszusatz    PLZ          Ort  \
0  Ennabeurer Weg          0          NaN  72535  Herold

In [28]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171805 entries, 0 to 171804
Data columns (total 68 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   ladestation_id                 171805 non-null  int64  
 1   Betreiber                      171805 non-null  object 
 2   BetreiberBereinigt             171803 non-null  object 
 3   BetreiberKategorie             171805 non-null  object 
 4   Strasse                        171805 non-null  object 
 5   Hausnummer                     171805 non-null  object 
 6   Adresszusatz                   29126 non-null   object 
 7   PLZ                            171805 non-null  int64  
 8   Ort                            171805 non-null  object 
 9   Bundesland                     171805 non-null  object 
 10  KreisKreisfreieStadt           171805 non-null  object 
 11  Gemeinde                       171805 non-null  object 
 12  ARS_left                      

,ladestation_id,Betreiber,BetreiberBereinigt,BetreiberKategorie,Strasse,Hausnummer,Adresszusatz,PLZ,Ort,Bundesland,...,SN_K,SN_V1,SN_V2,SN_G,FK_S3,NUTS,ARS_0,AGS_0,WSK,DLM_ID
0,1,Albwerk Elektro- und Kommunikationstechnik GmbH,Albwerk Elektro- und Kommunikationstechnik,Sonstiges,Ennabeurer Weg,0,NaN,72535,Heroldstatt,Baden-Württemberg,...,25,0,0,0,R,DE145,84250000000,8425000,2011-01-01,DEBKGDL20000E15K
1,1,Albwerk Elektro- und Kommunikationstechnik GmbH,Albwerk Elektro- und Kommunikationstechnik,Sonstiges,Ennabeurer Weg,0,NaN,72535,Heroldstatt,Baden-Württemberg,...,25,0,0,0,R,DE145,84250000000,8425000,2011-01-01,DEBKGDL20000E15K
2,2,smopi® - Multi Chargepoint Solution GmbH,smopi - Multi Chargepoint Solution,Sonstiges,Albstraße,15,NaN,72535,Heroldstatt,Baden-Württemberg,...,25,0,0,0,R,DE145,84250000000,8425000,2011-01-01,DEBKGDL20000E15K
3,2,smopi® - Multi Chargepoint Solution GmbH,smopi - Multi Chargepoint Solution,Sonstiges,Albstraße,15,NaN,72535,Heroldstatt,Baden-Württemberg,...,25,0,0,0,R,DE145,84250000000,8425000,2011-01-01,DEBKGDL20000E15K
4,2,smopi® - Multi Chargepoint Solution GmbH,smopi - Multi Chargepoint Solution,Sonstiges,Albstraße,15,NaN,72535,Heroldstatt,Baden-Württemberg,...,25,0,0,0,R,DE145,84250000000,8425000,2011-01-01,DEBKGDL20000E15K


In [29]:
df.shape

(171805, 68)